In [12]:
#Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("wordCount")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

24/06/11 16:48:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [13]:
spark

In [16]:
sc = spark.sparkContext

In [17]:
sc

<SparkContext master=local appName=wordCount>

In [27]:
# Create DataFrames
df1 = spark.range(1, 1000, 2)
df2 = spark.range(1, 1000, 2)

# Repartition the DataFrames
step1 = df1.repartition(5)
step2 = df2.repartition(6)

# Transformations
step3 = step1.selectExpr("id * 5 as id")
step4 = step3.join(step2, ["id"])
step5 = step1.selectExpr("sum(id) as sum_id")

# Show the result of step5
step5.collect()

[Row(sum_id=250000)]

In [29]:
step5.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(3) HashAggregate(keys=[], functions=[sum(id#100L)])
   +- ShuffleQueryStage 1
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=221]
         +- *(2) HashAggregate(keys=[], functions=[partial_sum(id#100L)])
            +- ShuffleQueryStage 0
               +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=200]
                  +- *(1) Range (1, 1000, step=2, splits=1)
+- == Initial Plan ==
   HashAggregate(keys=[], functions=[sum(id#100L)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=192]
      +- HashAggregate(keys=[], functions=[partial_sum(id#100L)])
         +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=188]
            +- Range (1, 1000, step=2, splits=1)


